# Plotting of explicit ion results

The plotting is done by accessing the results from files produced by the simulations, both thoes with tempering and without, to get the full potential of mean force over the whole range of offsets. 
Note that you might have to change workdir and how you access your files depending on how you have stored them. 

For the inline plotting adding the salt concentration on the curve, Matplotlib labe lines is used (Cadiou, C. (2022). Matplotlib label lines (v0.5.1). Zenodo. https://doi.org/10.5281/zenodo.7428071).

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os.path
import os
import json
from labellines import labelLine


workdir = '/home/isabel/Documents/SI-betalac-lactoferrin-main/tempering'
conc_list = [10, 15, 20, 25]
offset = np.linspace(50,125,76) 

In [11]:
def get_forces(offs, cs):
    ''' extract force from out.json as a function of offset '''
    forces = []
    for o in offs:
        if os.path.exists('Off' + o):
            if os.path.isfile('Off'+ o+'/mpi0.out.json'):
                with open('Off'+ o+'/mpi0.out.json') as f:
                    j = json.load(f) # --> dict
                    if j['analysis'][0]["virtualtranslate"]["force"] is not None:
                        f = j['analysis'][0]["virtualtranslate"]["force"]
                        forces.append(-f)
                    else:
                        forces.append(0)
            else:
                with open('Off'+ o+'/out.json') as f:
                    j = json.load(f) # --> dict
                    if j['analysis'][0]["virtualtranslate"]["force"] is not None:
                        f = j['analysis'][0]["virtualtranslate"]["force"]
                        forces.append(-f)
    return forces
     

def plotForce(ax, offsets, dL, style, col, cs, label, Pdown=0):    
    r = offsets
    dirs = map(str, offsets)
    f = get_forces(dirs, cs)
    pmf = np.cumsum(f)*dL # dL is difference between two r
    pmf = pmf -pmf[-1]  - Pdown # Pdown adjust the pmf in y
    r_Arr = np.array(r)
    ax.plot(r_Arr,pmf, style, color = col, label=label)
    
    return pmf

In [ ]:
plt.rcParams.update({'font.size': 18, 'figure.figsize': [10, 10.0]})   
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
my_cmap = ['#ff8600', '#0032cf', '#c6005f', '#000000', '#74895d']


for itr, conc in enumerate(conc_list):
    print(conc)
    %cd -q $workdir/cosmos_v4/data_$conc
    !pwd
    plotForce(ax, offset, 1, '-', my_cmap[itr], conc, label=f'Grand canonical explicit salt, {conc}mM', Pdown =0)

labelLines(plt.gca().get_lines(), xvals=[58, 58, 60, 62, 75] ,fontsize=14, color="k")

plt.xlabel('Protein-protein mass center separation, $R$/Å')
plt.ylabel('Interaction free energy, $w(R) / k_BT$')
plt.title('Interaction free energy of BLG and LF with explicit salt')
plt.ylim(-18, 1)
plt.legend(frameon=False, loc=0)
plt.show()